# SmolVLA 训练（Google Colab）

该笔记本负责从 Google Drive 同步项目文件、安装依赖并启动 `train_model.py`。

## 前置准备
- 已将 `lerobot/`、`train_model.py`、`smolvla_omy.yaml` 以及数据集（例如 `demo_data_language/`）上传至 Google Drive 的同一目录。
- 确保在 Colab 运行前，Drive 中的这些文件已经更新。
- 如果需要同步 WandB，请准备好 `WANDB_API_KEY`。若不使用 WandB，可在后续步骤关闭。

In [ ]:
#@title 挂载 Google Drive
from google.colab import drive

drive.mount("/content/drive", force_remount=True)


## 配置路径并同步文件
请在下方填入 Google Drive 中保存项目的目录。系统会将需要的文件拷贝到 Colab 的本地缓存中，以避免训练时频繁访问 Drive。

In [ ]:
#@title 设置目录并同步数据
from pathlib import Path
import os
import shutil
import subprocess

DRIVE_PROJECT_ROOT = "/content/drive/MyDrive/path_to_project"  #@param {type:"string"}
LOCAL_WORKDIR = "/content/smolvla_workspace"  #@param {type:"string"}
CACHE_ROOT = "/content/smolvla_cache"  #@param {type:"string"}

drive_root = Path(DRIVE_PROJECT_ROOT)
if not drive_root.exists():
    raise FileNotFoundError(f"未找到 Drive 目录: {drive_root}. 请修改 DRIVE_PROJECT_ROOT 并重新运行本单元。")

local_root = Path(LOCAL_WORKDIR)
local_root.mkdir(parents=True, exist_ok=True)

cache_root = Path(CACHE_ROOT)
cache_root.mkdir(parents=True, exist_ok=True)

env_cache_dirs = {
    "HF_HOME": cache_root / "huggingface",
    "WANDB_DIR": cache_root / "wandb",
    "WANDB_CONFIG_DIR": cache_root / "wandb",
    "XDG_CACHE_HOME": cache_root / "xdg",
}
for var, path in env_cache_dirs.items():
    path.mkdir(parents=True, exist_ok=True)
    os.environ[var] = str(path)

os.environ["LOCAL_WORKDIR"] = str(local_root)

# 默认关闭 WandB，需要时可手动设置 WANDB_DISABLED=false
os.environ.setdefault("WANDB_DISABLED", "true")

items_to_copy = ["lerobot", "train_model.py", "smolvla_omy.yaml", "demo_data_language"]
for item in items_to_copy:
    src = drive_root / item
    if not src.exists():
        print(f"⚠️ 未在 Drive 中找到 {src}，已跳过。")
        continue
    dst = local_root / item
    if src.is_dir():
        dst.mkdir(parents=True, exist_ok=True)
        subprocess.run(["rsync", "-a", "--info=progress2", str(src) + "/", str(dst) + "/"], check=True)
    else:
        dst.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(src, dst)

print("同步完成。当前工作目录:", local_root)


## 安装 Python 依赖
默认按 `lerobot[smolvla]` 安装。如需额外依赖，可在此单元后追加 `pip install`。

In [ ]:
#@title 安装项目依赖（可能耗时较长）
import os
import subprocess
import sys
from pathlib import Path

workspace = Path(os.environ["LOCAL_WORKDIR"])
lerobot_path = workspace / "lerobot"
if not lerobot_path.exists():
    raise FileNotFoundError(f"预期在 {lerobot_path} 找到 lerobot 目录。请确认已同步文件。")

subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "setuptools", "wheel"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-e", f"{lerobot_path}[smolvla]"], check=True)


## （可选）检查 GPU 与 Torch
确保 Colab 分配到了 GPU，并且 PyTorch 能识别。

In [ ]:
#@title 检查 GPU 与 Torch 版本
import torch

print('Torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('当前 GPU:', torch.cuda.get_device_name(0))
!nvidia-smi


## 启动训练
默认使用同步下来的 `smolvla_omy.yaml`，如需修改可在运行前调整。

In [ ]:
#@title 运行训练脚本
import os
import subprocess
import sys
from pathlib import Path

workspace = Path(os.environ["LOCAL_WORKDIR"])
config_path = workspace / "smolvla_omy.yaml"
train_script = workspace / "train_model.py"

if not config_path.exists():
    raise FileNotFoundError(f"找不到配置文件 {config_path}")
if not train_script.exists():
    raise FileNotFoundError(f"找不到训练脚本 {train_script}")

print("使用配置:", config_path)
print("开始训练...")
subprocess.run([sys.executable, str(train_script), "--config_path", str(config_path)], check=True)
